In [1]:
pip install sqlalchemy

   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/2.1 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/2.1 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/2.1 MB ? eta -:--:--
   ---------- ----------------------------- 0.5/2.1 MB 465.0 kB/s eta 0:00:04
   ---------- ----------------------------- 0.5/2.1 MB 465.0 kB/s eta 0:00:04
   --------------- ------------------------ 0.8/2.1 MB 496.4 kB/s eta 0:00:03
   --------------- ------------------------ 0.8/2.1 MB 496.4 kB/s eta 0:00:03
   --------------- ------------------------ 0.8/2.1 MB 496.4 kB/s eta 0:00:03
   -------------------- ------------------- 1.0/2.1 MB 461.7 kB/s eta 0:00:03
   -------------------- ------------------- 1.0/2.1 MB 461.7 kB/s eta 0:00:03
   -------------------- ----------------

In [2]:
pip install pyodbc pandas mlxtend 

   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
    --------------------------------------- 0.3/11.5 MB ? eta -:--:--
    --------------------------------------- 0.3/11.5 MB ? eta -:--:--
    --------------------------------------- 0.3/11.5 MB ? eta -:--:--
    --------------------------------------- 0.3/11.5 MB ? eta -:--:--
    --------------------------------------- 0.3/11.5 MB ? eta -:--:--
    --------------------------------------- 0.3/11.5 MB ? eta -:--:--
   - ---------------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [3]:
import pyodbc

# Define the connection string
conn_str = (
    r'DRIVER={ODBC Driver 17 for SQL Server};'
    r'SERVER=DESKTOP-JM83MH2;'
    r'DATABASE=CompanyX;'
    r'Trusted_Connection=yes;'
)

print(conn_str)

# Establish the connection
conn = pyodbc.connect(conn_str)

# Create a cursor object
cursor = conn.cursor()

# Test the connection by executing a simple query
cursor.execute("SELECT @@VERSION")

# Fetch and print the result
row = cursor.fetchone()
print(row[0])

# Close the cursor and connection


DRIVER={ODBC Driver 17 for SQL Server};SERVER=DESKTOP-JM83MH2;DATABASE=CompanyX;Trusted_Connection=yes;
Microsoft SQL Server 2022 (RTM) - 16.0.1000.6 (X64) 
	Oct  8 2022 05:58:25 
	Copyright (C) 2022 Microsoft Corporation
	Developer Edition (64-bit) on Windows 10 Pro 10.0 <X64> (Build 26100: )



In [4]:
import pandas as pd

In [5]:
querySalesFact = "SELECT * FROM dbo.SalesFact"
df_SalesFact = pd.read_sql(querySalesFact, conn)

queryProductDim = "Select * from dbo.ProductDim"
queryDateDim = "Select * from dbo.DateDim"
queryPromotionDim = "Select * from dbo.PromotionDim"
queryTerritoryDim = "Select * from dbo.TerritoryDim"
queryInventoryDim = "Select * from dbo.InventoryDim"

df_ProductDim = pd.read_sql(queryProductDim,conn)
df_DateDim = pd.read_sql(queryDateDim,conn)
df_PromotionDim = pd.read_sql(queryPromotionDim,conn)
df_TerritoryDim = pd.read_sql(queryTerritoryDim,conn)
df_InventoryDim = pd.read_sql(queryInventoryDim,conn)

C:\Users\lenovo\AppData\Local\Temp\ipykernel_20132\2369303576.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_SalesFact = pd.read_sql(querySalesFact, conn)
C:\Users\lenovo\AppData\Local\Temp\ipykernel_20132\2369303576.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_ProductDim = pd.read_sql(queryProductDim,conn)
C:\Users\lenovo\AppData\Local\Temp\ipykernel_20132\2369303576.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_DateDim = pd.read_sql(queryDateDim,conn)
C:\Users\lenovo\AppData\Local\Temp\ipykern

In [6]:
import numpy as np 
from mlxtend.frequent_patterns import apriori, association_rules 

df_extracted = df_SalesFact[['TransactionID', 'ProductID', 'QuantitySold']].fillna(0)
df_extracted = (df_extracted.groupby(['TransactionID', 'ProductID'])['QuantitySold']
    .sum().unstack().reset_index().fillna(0) 
    .set_index('TransactionID')) 

df_extracted.head()

ProductID,707,708,709,710,711,712,713,714,715,716,...,989,990,991,992,993,994,996,997,998,999
TransactionID,,,,,,,,,,,,,,,,,,,,,
43659,0.0,0.0,6.0,0.0,4.0,2.0,0.0,3.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43660,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43661,0.0,5.0,0.0,0.0,2.0,4.0,0.0,0.0,4.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43662,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43663,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
def hot_encode(x): 
    if(x<= 0): 
        return 0
    if(x>= 1):
        return 1

df_extracted = df_extracted.applymap(hot_encode)
df_extracted.fillna(0, inplace=True)

df_extracted.fillna(0)
frq_items = apriori(df_extracted, min_support = 0.01, use_colnames = True) 
  
rules = association_rules(frq_items, metric ="lift", min_threshold = 0, num_itemsets = len(frq_items)) 
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 
print(rules) 

C:\Users\lenovo\AppData\Local\Temp\ipykernel_20132\1534686032.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_extracted = df_extracted.applymap(hot_encode)
c:\Users\lenovo\AppData\Local\Programs\Python\Python313\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


           antecedents      consequents  antecedent support  \
14395  (864, 707, 708)       (883, 884)            0.010025   
15355  (864, 707, 711)       (883, 884)            0.010025   
16165  (864, 707, 715)       (883, 884)            0.010057   
16375  (707, 876, 715)       (883, 884)            0.010057   
16555  (864, 707, 876)       (883, 884)            0.010025   
...                ...              ...                 ...   
775              (870)       (859, 707)            0.149666   
1189             (870)       (859, 708)            0.149666   
3294             (870)  (707, 708, 715)            0.149666   
3044             (870)  (707, 708, 711)            0.149666   
3798             (870)  (707, 715, 711)            0.149666   

       consequent support   support  confidence       lift  representativity  \
14395            0.012100  0.010025     1.00000  82.646438               1.0   
15355            0.012100  0.010025     1.00000  82.646438               1.0   
161

In [9]:
product_id_to_name = df_ProductDim.set_index('ProductID')['ProductName'].to_dict()

# Function to replace ProductID with ProductName in itemsets
def replace_ids_with_names(itemset):
    return [product_id_to_name.get(item, item) for item in itemset]

# Apply the function to the antecedents and consequents columns
rules['antecedents'] = rules['antecedents'].apply(replace_ids_with_names)
rules['consequents'] = rules['consequents'].apply(replace_ids_with_names)

In [11]:
from sqlalchemy import create_engine
engine = create_engine(f"mssql+pyodbc:///?odbc_connect={conn_str}")

recRules = rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']]

recRules['antecedents'] = recRules['antecedents'].apply(lambda x: '; '.join(list(x)))
recRules['consequents'] = recRules['consequents'].apply(lambda x: '; '.join(list(x)))
recRules

recRules.to_sql('RecRule', engine, if_exists='append', index=False)

C:\Users\lenovo\AppData\Local\Temp\ipykernel_20132\2801520011.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recRules['antecedents'] = recRules['antecedents'].apply(lambda x: '; '.join(list(x)))
C:\Users\lenovo\AppData\Local\Temp\ipykernel_20132\2801520011.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recRules['consequents'] = recRules['consequents'].apply(lambda x: '; '.join(list(x)))


233

In [12]:
# Create a dictionary to map ProductID to ProductName
product_id_to_name = df_ProductDim.set_index('ProductID')['ProductName'].to_dict()

# Function to replace ProductID with ProductName in itemsets
def replace_ids_with_names(itemset):
    return frozenset([product_id_to_name.get(item, item) for item in itemset])

# Apply the function to the antecedents and consequents columns
rules['antecedents'] = rules['antecedents'].apply(replace_ids_with_names)
rules['consequents'] = rules['consequents'].apply(replace_ids_with_names)

rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
14395,"(Classic Vest, S, Sport-100 Helmet, Black, Spo...","(Short-Sleeve Classic Jersey, L, Short-Sleeve ...",0.010025,0.012100,0.010025,1.00000,82.646438,1.0,0.009903,inf,0.997904,0.828496,1.000000,0.914248
15355,"(Classic Vest, S, Sport-100 Helmet, Red, Sport...","(Short-Sleeve Classic Jersey, L, Short-Sleeve ...",0.010025,0.012100,0.010025,1.00000,82.646438,1.0,0.009903,inf,0.997904,0.828496,1.000000,0.914248
16165,"(Classic Vest, S, Long-Sleeve Logo Jersey, L, ...","(Short-Sleeve Classic Jersey, L, Short-Sleeve ...",0.010057,0.012100,0.010057,1.00000,82.646438,1.0,0.009935,inf,0.997936,0.831135,1.000000,0.915567
16375,"(Long-Sleeve Logo Jersey, L, Sport-100 Helmet,...","(Short-Sleeve Classic Jersey, L, Short-Sleeve ...",0.010057,0.012100,0.010057,1.00000,82.646438,1.0,0.009935,inf,0.997936,0.831135,1.000000,0.915567
16555,"(Classic Vest, S, Sport-100 Helmet, Red, Hitch...","(Short-Sleeve Classic Jersey, L, Short-Sleeve ...",0.010025,0.012100,0.010025,1.00000,82.646438,1.0,0.009903,inf,0.997904,0.828496,1.000000,0.914248
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
775,(Water Bottle - 30 oz.),"(Sport-100 Helmet, Red, Half-Finger Gloves, M)",0.149666,0.019251,0.010025,0.06698,3.479270,1.0,0.007143,1.051155,0.838004,0.063090,0.048665,0.293855
1189,(Water Bottle - 30 oz.),"(Sport-100 Helmet, Black, Half-Finger Gloves, M)",0.149666,0.019347,0.010025,0.06698,3.462046,1.0,0.007129,1.051052,0.836323,0.063052,0.048572,0.292566
3294,(Water Bottle - 30 oz.),"(Sport-100 Helmet, Black, Sport-100 Helmet, Re...",0.149666,0.024295,0.010025,0.06698,2.756898,1.0,0.006388,1.045748,0.749439,0.061149,0.043747,0.239797
3044,(Water Bottle - 30 oz.),"(Sport-100 Helmet, Black, Sport-100 Helmet, Re...",0.149666,0.024359,0.010025,0.06698,2.749672,1.0,0.006379,1.045680,0.748318,0.061125,0.043685,0.239256


In [13]:
# Filter rules with only one element in both antecedents and consequents
filtered_rules = rules[rules['antecedents'].apply(lambda x: len(x) == 1) & rules['consequents'].apply(lambda x: len(x) == 1)]
filtered_rules['antecedents'] = filtered_rules['antecedents'].apply(replace_ids_with_names)
filtered_rules['consequents'] = filtered_rules['consequents'].apply(replace_ids_with_names)

print(filtered_rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']])
extractedRules = filtered_rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']]



                 antecedents                       consequents   support  \
125      (Women's Tights, L)                    (AWC Logo Cap)  0.013217   
326      (Women's Tights, S)               (Women's Tights, L)  0.012611   
173      (Women's Tights, L)      (Long-Sleeve Logo Jersey, L)  0.013153   
82     (Men's Bib-Shorts, M)          (Sport-100 Helmet, Blue)  0.010472   
123      (Women's Tights, S)                    (AWC Logo Cap)  0.012387   
..                       ...                               ...       ...   
365  (Water Bottle - 30 oz.)                 (Classic Vest, M)  0.011908   
389  (Water Bottle - 30 oz.)  (Short-Sleeve Classic Jersey, S)  0.011685   
384  (Water Bottle - 30 oz.)           (Fender Set - Mountain)  0.010440   
335  (Water Bottle - 30 oz.)           (Half-Finger Gloves, S)  0.010344   
159  (Water Bottle - 30 oz.)      (Long-Sleeve Logo Jersey, M)  0.010152   

     confidence       lift  
125    0.938776   8.699783  
326    0.936019  66.482816  


C:\Users\lenovo\AppData\Local\Temp\ipykernel_20132\3101147571.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_rules['antecedents'] = filtered_rules['antecedents'].apply(replace_ids_with_names)
C:\Users\lenovo\AppData\Local\Temp\ipykernel_20132\3101147571.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_rules['consequents'] = filtered_rules['consequents'].apply(replace_ids_with_names)


In [14]:


# Convert frozenset to string
extractedRules['antecedents'] = extractedRules['antecedents'].apply(lambda x: ', '.join(list(x)))
extractedRules['consequents'] = extractedRules['consequents'].apply(lambda x: ', '.join(list(x)))

extractedRules.to_sql('PairRule', engine, if_exists='append', index=False)

C:\Users\lenovo\AppData\Local\Temp\ipykernel_20132\359510155.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  extractedRules['antecedents'] = extractedRules['antecedents'].apply(lambda x: ', '.join(list(x)))
C:\Users\lenovo\AppData\Local\Temp\ipykernel_20132\359510155.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  extractedRules['consequents'] = extractedRules['consequents'].apply(lambda x: ', '.join(list(x)))


27

In [15]:
# Convert the StartDate and EndDate columns to datetime if they are not already
df_PromotionDim['StartDate'] = pd.to_datetime(df_PromotionDim['StartDate'])
df_PromotionDim['EndDate'] = pd.to_datetime(df_PromotionDim['EndDate'])

# Create a dictionary to store dataframes for each promotion period
promotion_dfs = {}

# Iterate through each promotion
for _, row in df_PromotionDim.iterrows():
    promotion_id = row['PromotionID']
    start_date = row['StartDate']
    end_date = row['EndDate']
    
    # Filter the SalesFact dataframe for the given promotion period
    promotion_df = df_SalesFact[(df_SalesFact['PromotionID'] == promotion_id) & 
                                (df_SalesFact['ModifiedDate'] >= start_date) & 
                                (df_SalesFact['ModifiedDate'] <= end_date)]
    
    # Store the dataframe in the dictionary
    promotion_dfs[promotion_id] = promotion_df

# Create a dataframe for transactions with no promotion
no_promotion_df = df_SalesFact[df_SalesFact['PromotionID'] == 1]

# Display the dataframes
for promotion_id, promotion_df in promotion_dfs.items():
    print(f"Promotion ID: {promotion_id}")
    print(promotion_df.head())

print("No Promotion DataFrame:")
print(no_promotion_df.head())

Promotion ID: 1
   SalesFactID  TransactionID  ProductID  TerritoryID  PromotionID  DateID  \
0            1          43659        776            5            1     652   
1            2          43659        776            5            1     652   
2            3          43659        777            5            1     652   
3            4          43659        777            5            1     652   
4            5          43659        778            5            1     652   

   QuantitySold  UnitPrice  TotalSaleAmount  DiscountApplied  \
0           1.0   2024.994         2024.994              0.0   
1           1.0   2024.994         2024.994              0.0   
2           3.0   2024.994         6074.982              0.0   
3           3.0   2024.994         6074.982              0.0   
4           1.0   2024.994         2024.994              0.0   

             ModifiedDate  InventoryID  
0 2014-02-08 10:01:36.827          686  
1 2014-02-08 10:01:36.827          687  
2 2014-

In [16]:
def process_dataframe(df):
    df_extracted = df[['TransactionID', 'ProductID', 'QuantitySold']].fillna(0)
    df_extracted = (df_extracted.groupby(['TransactionID', 'ProductID'])['QuantitySold']
                    .sum().unstack().reset_index().fillna(0)
                    .set_index('TransactionID'))
    return df_extracted

# Apply the function to each dataframe in promotion_dfs
for promotion_id in promotion_dfs:
    promotion_dfs[promotion_id] = process_dataframe(promotion_dfs[promotion_id])

# Apply the function to no_promotion_df
no_promotion_df_processed = process_dataframe(no_promotion_df)

In [17]:
for promotion_id in promotion_dfs:
    promotion_dfs[promotion_id] = promotion_dfs[promotion_id].applymap(hot_encode)


C:\Users\lenovo\AppData\Local\Temp\ipykernel_20132\7612559.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  promotion_dfs[promotion_id] = promotion_dfs[promotion_id].applymap(hot_encode)


In [18]:
no_promotion_df_processed = no_promotion_df_processed.applymap(hot_encode)

C:\Users\lenovo\AppData\Local\Temp\ipykernel_20132\1819978960.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  no_promotion_df_processed = no_promotion_df_processed.applymap(hot_encode)


In [19]:
frq_nopromotion_items = apriori(no_promotion_df_processed, min_support = 0.03, use_colnames=True)
nonpromotion_rules = association_rules(frq_nopromotion_items, metric = "lift", min_threshold=0, num_itemsets = len(frq_nopromotion_items))
nonpromotion_rules = nonpromotion_rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 

frq_promotion_items = {}
promotion_rules = {}
promotion_rules_id = []
for promotion_id in promotion_dfs:
    frq_promotion_items[promotion_id] = apriori(promotion_dfs[promotion_id], min_support=0.03, use_colnames=True)
    if (len(frq_promotion_items[promotion_id]) > 0):
        promotion_rules_id.append(promotion_id)
        promotion_rules[promotion_id] = association_rules(frq_promotion_items[promotion_id], metric = "lift", min_threshold=0, num_itemsets = len(frq_promotion_items[promotion_id]))
        promotion_rules[promotion_id].sort_values(['confidence', 'lift'], ascending =[False, False]) 

c:\Users\lenovo\AppData\Local\Programs\Python\Python313\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
c:\Users\lenovo\AppData\Local\Programs\Python\Python313\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
c:\Users\lenovo\AppData\Local\Programs\Python\Python313\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
c:\Users\lenovo\AppData\Local\Programs\Python\

In [20]:
for id in promotion_rules_id :
    print(promotion_rules[id])

  antecedents consequents  antecedent support  consequent support   support  \
0       (712)       (715)            0.106218            0.049436  0.032405   
1       (715)       (712)            0.049436            0.106218  0.032405   
2       (870)       (871)            0.150583            0.065817  0.054994   
3       (871)       (870)            0.065817            0.150583  0.054994   
4       (872)       (870)            0.055644            0.150583  0.049436   
5       (870)       (872)            0.150583            0.055644  0.049436   
6       (921)       (930)            0.100595            0.045373  0.031105   
7       (930)       (921)            0.045373            0.100595  0.031105   

   confidence      lift  representativity  leverage  conviction  \
0    0.305080  6.171192               1.0  0.027154    1.367874   
1    0.655490  6.171192               1.0  0.027154    2.594357   
2    0.365206  5.548789               1.0  0.045083    1.471632   
3    0.835556  5.548

In [21]:
# Create an empty list to store the results
results = []

# Iterate through each promotion rule
for promotion_id in promotion_rules_id:
    promotion_rule = promotion_rules[promotion_id]
    
    # Iterate through each row in the promotion rule
    for _, row in promotion_rule.iterrows():
        antecedents = row['antecedents']
        consequents = row['consequents']
        confidence = row['confidence']
        support = row['support']
        lift = row['lift']
        
        # Find the corresponding row in non_promotion_rules
        match = nonpromotion_rules[(nonpromotion_rules['antecedents'] == antecedents) & 
                                   (nonpromotion_rules['consequents'] == consequents)]
        
        if not match.empty:
            non_promotion_confidence = match.iloc[0]['confidence']
            non_promotion_support = match.iloc[0]['support']
            non_promotion_lift = match.iloc[0]['lift']
            
            # Check if the support in promotion_rules is higher
            if support > non_promotion_support :
                results.append({
                    'antecedents': antecedents,
                    'consequents': consequents,
                    'promotion_id': promotion_id,
                    'promotion_support': support,
                    'non_promotion_support': non_promotion_support,
                    'support_difference': support - non_promotion_support
                })

# Convert the results to a DataFrame
df_higher_support = pd.DataFrame(results)

# Display the DataFrame
print(df_higher_support)

Empty DataFrame
Columns: []
Index: []


In [22]:
print(df_SalesFact[1:30])

    SalesFactID  TransactionID  ProductID  TerritoryID  PromotionID  DateID  \
1             2          43659        776            5            1     652   
2             3          43659        777            5            1     652   
3             4          43659        777            5            1     652   
4             5          43659        778            5            1     652   
5             6          43659        778            5            1     652   
6             7          43659        771            5            1     652   
7             8          43659        771            5            1     652   
8             9          43659        772            5            1     652   
9            10          43659        772            5            1     652   
10           11          43659        773            5            1     652   
11           12          43659        773            5            1     652   
12           13          43659        774           

In [23]:
df_extracted = df_SalesFact[['TransactionID', 'ProductID', 'QuantitySold', 'ModifiedDate']].fillna(0)
df_extracted['ProductPromotionID'] = df_SalesFact['ProductID'].astype(str) + '_' + df_SalesFact['PromotionID'].astype(str)

df_extracted.where(df_extracted['ProductID'] == 877).dropna()

,TransactionID,ProductID,QuantitySold,ModifiedDate,ProductPromotionID
58534,51081.0,877.0,7.0,2014-02-08 10:01:36.827,877_1
58595,51083.0,877.0,4.0,2014-02-08 10:01:36.827,877_1
58677,51085.0,877.0,4.0,2014-02-08 10:01:36.827,877_1
58841,51090.0,877.0,6.0,2014-02-08 10:01:36.827,877_1
58967,51091.0,877.0,7.0,2014-02-08 10:01:36.827,877_1
...,...,...,...,...,...
189012,75094.0,877.0,1.0,2014-02-08 10:01:36.827,877_1
189027,75099.0,877.0,1.0,2014-02-08 10:01:36.827,877_1
189038,75104.0,877.0,1.0,2014-02-08 10:01:36.827,877_1
189055,75110.0,877.0,1.0,2014-02-08 10:01:36.827,877_1


In [24]:
df_extracted = df_SalesFact[['TransactionID', 'ProductID', 'QuantitySold', 'ModifiedDate']].fillna(0)
df_extracted['ProductPromotionID'] = df_SalesFact['ProductID'].astype(str) + '_' + df_SalesFact['PromotionID'].astype(str)


df_extracted = (df_extracted.groupby(['TransactionID', 'ProductPromotionID'])['QuantitySold']
    .sum().unstack().reset_index().fillna(0) 
    .set_index('TransactionID')) 

df_extracted.head()
df_extracted = df_extracted.applymap(hot_encode)
df_extracted.fillna(0, inplace=True)

frq_items = apriori(df_extracted, min_support = 0.01, use_colnames = True)


rules = association_rules(frq_items, metric ="lift", min_threshold = 0, num_itemsets = len(frq_items))
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False])

C:\Users\lenovo\AppData\Local\Temp\ipykernel_20132\2409831490.py:10: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_extracted = df_extracted.applymap(hot_encode)
c:\Users\lenovo\AppData\Local\Programs\Python\Python313\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [25]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
707,"(870_1, 878_1)",(871_1),0.010440,0.064649,0.010440,1.000000,15.468148,1.0,0.009765,inf,0.945219,0.161481,1.000000,0.580741
1878,"(711_1, 714_1, 712_1, 707_1, 715_1)",(708_1),0.012291,0.092105,0.011525,0.937662,10.180380,1.0,0.010393,14.564151,0.912994,0.124098,0.931338,0.531396
1428,"(715_1, 707_1, 711_1, 712_1)",(708_1),0.015771,0.092105,0.014781,0.937247,10.175871,1.0,0.013329,14.467749,0.916177,0.158779,0.930881,0.548866
824,"(859_1, 708_1, 712_1)",(707_1),0.011397,0.095042,0.010663,0.935574,9.843800,1.0,0.009580,14.046522,0.908771,0.111333,0.928808,0.523884
1148,"(859_1, 711_1, 712_1)",(708_1),0.011365,0.092105,0.010631,0.935393,10.155745,1.0,0.009584,14.052638,0.911898,0.114512,0.928839,0.525409
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,(870_1),(715_1),0.147910,0.048559,0.011908,0.080509,1.657985,1.0,0.004726,1.034748,0.465747,0.064522,0.033581,0.162871
206,(870_1),(859_1),0.147910,0.033330,0.011270,0.076193,2.285995,1.0,0.006340,1.046398,0.660205,0.066304,0.044340,0.207158
212,(870_1),(876_1),0.147910,0.023433,0.010950,0.074034,3.159360,1.0,0.007484,1.054647,0.802123,0.068272,0.051815,0.270668
709,(870_1),"(871_1, 878_1)",0.147910,0.012515,0.010440,0.070581,5.639787,1.0,0.008589,1.062475,0.965495,0.069604,0.058802,0.452382


In [26]:
def clean_itemset(itemset):
    # Split items into prefix and suffix and group by prefix
    prefix_map = {}
    for item in itemset:
        prefix, suffix = item.split('_')
        suffix = int(suffix)
        if prefix not in prefix_map or suffix < prefix_map[prefix][1]:
            prefix_map[prefix] = (item, suffix)
    
    # Extract only the items with the smallest suffix per prefix
    return {v[0] for v in prefix_map.values()}

# Apply the function to clean the itemsets
frq_items['itemsets'] = frq_items['itemsets'].apply(clean_itemset)



In [27]:

rules = association_rules(frq_items, metric ="lift", min_threshold = 0, num_itemsets = len(frq_items))
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False])

print(rules)

                              antecedents     consequents  antecedent support  \
707                        (870_1, 878_1)         (871_1)            0.010440   
1879  (711_1, 714_1, 712_1, 707_1, 715_1)         (708_1)            0.012291   
1429         (715_1, 712_1, 711_1, 707_1)         (708_1)            0.015771   
825                 (859_1, 708_1, 712_1)         (707_1)            0.011397   
1148                (859_1, 711_1, 712_1)         (708_1)            0.011365   
...                                   ...             ...                 ...   
90                                (870_1)         (715_1)            0.147910   
206                               (870_1)         (859_1)            0.147910   
212                               (870_1)         (876_1)            0.147910   
709                               (870_1)  (871_1, 878_1)            0.147910   
216                               (870_1)         (878_1)            0.147910   

      consequent support   

In [28]:
def remove_no_promotion(rule):
    for id, row in rule.iterrows():
        antecedents = list(row['antecedents'])[0]
        consequents = list(row['consequents'])[0]
        _, a_suffix = antecedents.split('_')
        _, c_suffix = consequents.split('_')
        if a_suffix == '1' and c_suffix == '1':
            rule.drop(index=id, inplace=True)

# Remove rules with no promotion
remove_no_promotion(rules)

print(rules)

        

    antecedents consequents  antecedent support  consequent support   support  \
234     (873_2)     (921_1)            0.026945            0.098809  0.010599   
235     (921_1)     (873_2)            0.098809            0.026945  0.010599   

     confidence      lift  representativity  leverage  conviction  \
234    0.393365  3.981056               1.0  0.007937    1.485557   
235    0.107270  3.981056               1.0  0.007937    1.089976   

     zhangs_metric   jaccard  certainty  kulczynski  
234       0.769546  0.092043   0.326852    0.250317  
235       0.830912  0.092043   0.082549    0.250317  


In [29]:
df_InventoryDim

,InventoryID,ProductID,Shelf,Bin,Quantity,LocationName,ModifiedDate,LocationID
0,1,1,A,1,408,Tool Crib,2014-08-08,1
1,2,1,B,5,324,Miscellaneous Storage,2014-08-08,6
2,3,1,A,5,353,Subassembly,2014-08-08,50
3,4,2,A,2,427,Tool Crib,2014-08-08,1
4,5,2,B,1,318,Miscellaneous Storage,2014-08-08,6
...,...,...,...,...,...,...,...,...
1064,1065,997,N/A,0,30,Final Assembly,2013-04-30,60
1065,1066,998,N/A,0,99,Finished Goods Storage,2013-04-30,7
1066,1067,998,N/A,0,56,Final Assembly,2013-04-30,60
1067,1068,999,N/A,0,78,Finished Goods Storage,2013-04-30,7


In [30]:
df_inventory = df_InventoryDim[['ProductID', 'Quantity']]

df_inventory = df_inventory.groupby('ProductID').sum().reset_index()
print(df_inventory)

     ProductID  Quantity
0            1      1085
1            2      1109
2            3      1352
3            4      1322
4          316      1361
..         ...       ...
427        995       816
428        996       970
429        997       153
430        998       155
431        999       194

[432 rows x 2 columns]


In [31]:
def identify_excess_inventory(inventory, sales_data, horizon=30):
    # Calculate average daily sales
    daily_sales = sales_data.groupby('ProductID')['QuantitySold'].sum() / sales_data.groupby('ProductID')['QuantitySold'].count()
    
    print(daily_sales)

    # Identify excess inventory
    excess_inventory = {}
    for product, stock_level in inventory[['ProductID', 'Quantity']].values:
        forecasted_demand = daily_sales.get(product, 0) * horizon
        if stock_level > forecasted_demand * 1.3:
            excess_inventory[product] = int(stock_level - forecasted_demand)
    
    return excess_inventory

excess_inventory = identify_excess_inventory(inventory=df_inventory, sales_data=df_SalesFact, horizon=30)

ProductID
707    2.024170
708    2.158986
709    5.888298
710    2.045455
711    2.170513
         ...   
994    2.520000
996    2.490826
997    1.402542
998    2.267831
999    1.918455
Name: QuantitySold, Length: 210, dtype: float64


In [32]:
# Get the unique ProductIDs from df_SalesFact
salesfact_product_ids = df_SalesFact['ProductID'].unique()

# Filter excess_inventory to only include ProductIDs present in df_SalesFact
filtered_excess_inventory = {product_id: quantity for product_id, quantity in excess_inventory.items() if product_id in salesfact_product_ids}



filtered_excess_inventory

{np.int64(707): 227,
 np.int64(708): 259,
 np.int64(710): 154,
 np.int64(711): 150,
 np.int64(712): 214,
 np.int64(713): 114,
 np.int64(714): 90,
 np.int64(715): 95,
 np.int64(716): 169,
 np.int64(747): 765,
 np.int64(748): 664,
 np.int64(749): 94,
 np.int64(750): 186,
 np.int64(751): 103,
 np.int64(752): 91,
 np.int64(753): 121,
 np.int64(754): 70,
 np.int64(755): 72,
 np.int64(756): 68,
 np.int64(757): 73,
 np.int64(758): 86,
 np.int64(759): 97,
 np.int64(760): 86,
 np.int64(761): 59,
 np.int64(762): 39,
 np.int64(763): 135,
 np.int64(764): 64,
 np.int64(765): 46,
 np.int64(766): 93,
 np.int64(767): 122,
 np.int64(768): 74,
 np.int64(769): 67,
 np.int64(770): 130,
 np.int64(771): 69,
 np.int64(772): 72,
 np.int64(773): 80,
 np.int64(774): 89,
 np.int64(775): 67,
 np.int64(776): 110,
 np.int64(777): 64,
 np.int64(778): 76,
 np.int64(779): 71,
 np.int64(780): 158,
 np.int64(781): 77,
 np.int64(782): 116,
 np.int64(783): 80,
 np.int64(784): 77,
 np.int64(785): 43,
 np.int64(786): 118,
 

In [33]:
len(filtered_excess_inventory)

192

In [34]:
df_extracted = df_SalesFact[['TransactionID', 'ProductID', 'QuantitySold']].fillna(0)
df_extracted = (df_extracted.groupby(['TransactionID', 'ProductID'])['QuantitySold']
    .sum().unstack().reset_index().fillna(0) 
    .set_index('TransactionID')) 

df_extracted = df_extracted.applymap(hot_encode)
df_extracted.fillna(0, inplace=True)

df_extracted.fillna(0)
frq_items = apriori(df_extracted, min_support = 0.01, use_colnames = True) 
  
rules = association_rules(frq_items, metric ="lift", min_threshold = 0, num_itemsets = len(frq_items)) 
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 
print(rules.head()) 

C:\Users\lenovo\AppData\Local\Temp\ipykernel_20132\1538987399.py:6: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_extracted = df_extracted.applymap(hot_encode)
c:\Users\lenovo\AppData\Local\Programs\Python\Python313\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


           antecedents consequents  antecedent support  consequent support  \
14395  (864, 707, 708)  (883, 884)            0.010025              0.0121   
15355  (864, 707, 711)  (883, 884)            0.010025              0.0121   
16165  (864, 707, 715)  (883, 884)            0.010057              0.0121   
16375  (707, 876, 715)  (883, 884)            0.010057              0.0121   
16555  (864, 707, 876)  (883, 884)            0.010025              0.0121   

        support  confidence       lift  representativity  leverage  \
14395  0.010025         1.0  82.646438               1.0  0.009903   
15355  0.010025         1.0  82.646438               1.0  0.009903   
16165  0.010057         1.0  82.646438               1.0  0.009935   
16375  0.010057         1.0  82.646438               1.0  0.009935   
16555  0.010025         1.0  82.646438               1.0  0.009903   

       conviction  zhangs_metric   jaccard  certainty  kulczynski  
14395         inf       0.997904  0.828496

In [35]:
def extract_product_ids(itemset):
    return set(itemset)

# Extract ProductIDs from antecedents and consequents
rules['antecedent_ids'] = rules['antecedents'].apply(extract_product_ids)
rules['consequent_ids'] = rules['consequents'].apply(extract_product_ids)

# Filter rows where ProductIDs are in filtered_excess_inventory
filtered_rules = rules[
    rules['antecedent_ids'].apply(lambda x: any(int(pid) in filtered_excess_inventory for pid in x)) |
    rules['consequent_ids'].apply(lambda x: any(int(pid) in filtered_excess_inventory for pid in x))
]

# Drop the temporary columns
filtered_rules = filtered_rules.drop(columns=['antecedent_ids', 'consequent_ids'])

filtered_rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
14395,"(864, 707, 708)","(883, 884)",0.010025,0.012100,0.010025,1.00000,82.646438,1.0,0.009903,inf,0.997904,0.828496,1.000000,0.914248
15355,"(864, 707, 711)","(883, 884)",0.010025,0.012100,0.010025,1.00000,82.646438,1.0,0.009903,inf,0.997904,0.828496,1.000000,0.914248
16165,"(864, 707, 715)","(883, 884)",0.010057,0.012100,0.010057,1.00000,82.646438,1.0,0.009935,inf,0.997936,0.831135,1.000000,0.915567
16375,"(707, 876, 715)","(883, 884)",0.010057,0.012100,0.010057,1.00000,82.646438,1.0,0.009935,inf,0.997936,0.831135,1.000000,0.915567
16555,"(864, 707, 876)","(883, 884)",0.010025,0.012100,0.010025,1.00000,82.646438,1.0,0.009903,inf,0.997904,0.828496,1.000000,0.914248
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
775,(870),"(859, 707)",0.149666,0.019251,0.010025,0.06698,3.479270,1.0,0.007143,1.051155,0.838004,0.063090,0.048665,0.293855
1189,(870),"(859, 708)",0.149666,0.019347,0.010025,0.06698,3.462046,1.0,0.007129,1.051052,0.836323,0.063052,0.048572,0.292566
3294,(870),"(707, 708, 715)",0.149666,0.024295,0.010025,0.06698,2.756898,1.0,0.006388,1.045748,0.749439,0.061149,0.043747,0.239797
3044,(870),"(707, 708, 711)",0.149666,0.024359,0.010025,0.06698,2.749672,1.0,0.006379,1.045680,0.748318,0.061125,0.043685,0.239256


In [36]:
# Filter rules where either antecedents or consequents intersect with filtered_excess_inventory
filtered_rules = rules[
    rules['antecedent_ids'].apply(lambda x: any(int(pid) in filtered_excess_inventory for pid in x)) |
    rules['consequent_ids'].apply(lambda x: any(int(pid) in filtered_excess_inventory for pid in x))
]

# Keep only IDs in filtered_excess_inventory for antecedents and consequents
filtered_rules['antecedent_ids'] = filtered_rules['antecedent_ids'].apply(
    lambda x: {int(pid) for pid in x if int(pid) in filtered_excess_inventory}
)
filtered_rules['consequent_ids'] = filtered_rules['consequent_ids'].apply(
    lambda x: {int(pid) for pid in x if int(pid) in filtered_excess_inventory}
)

# Drop rules where antecedents or consequents become empty after filtering
filtered_rules = filtered_rules[
    (filtered_rules['consequent_ids'].apply(len) > 0)
]

filtered_rules


C:\Users\lenovo\AppData\Local\Temp\ipykernel_20132\3959816356.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_rules['antecedent_ids'] = filtered_rules['antecedent_ids'].apply(
C:\Users\lenovo\AppData\Local\Temp\ipykernel_20132\3959816356.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_rules['consequent_ids'] = filtered_rules['consequent_ids'].apply(


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski,antecedent_ids,consequent_ids
2822,"(878, 870)",(871),0.010440,0.064649,0.010440,1.000000,15.468148,1.0,0.009765,inf,0.945219,0.161481,1.000000,0.580741,"{870, 878}",{871}
7079,"(708, 883, 884)",(715),0.011014,0.052102,0.010982,0.997101,19.137383,1.0,0.010408,327.024710,0.958301,0.210655,0.996942,0.603943,{708},{715}
24929,"(884, 708, 877)","(883, 715)",0.010982,0.012611,0.010950,0.997093,79.068215,1.0,0.010812,339.661974,0.998317,0.866162,0.997056,0.932724,{708},{715}
7038,"(708, 884, 877)",(715),0.010982,0.052102,0.010950,0.997093,19.137221,1.0,0.010378,326.076813,0.958270,0.210043,0.996933,0.603632,{708},{715}
24923,"(884, 883, 708, 877)",(715),0.010982,0.052102,0.010950,0.997093,19.137221,1.0,0.010378,326.076813,0.958270,0.210043,0.996933,0.603632,{708},{715}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
775,(870),"(859, 707)",0.149666,0.019251,0.010025,0.066980,3.479270,1.0,0.007143,1.051155,0.838004,0.063090,0.048665,0.293855,{870},{707}
1189,(870),"(859, 708)",0.149666,0.019347,0.010025,0.066980,3.462046,1.0,0.007129,1.051052,0.836323,0.063052,0.048572,0.292566,{870},{708}
3294,(870),"(707, 708, 715)",0.149666,0.024295,0.010025,0.066980,2.756898,1.0,0.006388,1.045748,0.749439,0.061149,0.043747,0.239797,{870},"{707, 708, 715}"
3044,(870),"(707, 708, 711)",0.149666,0.024359,0.010025,0.066980,2.749672,1.0,0.006379,1.045680,0.748318,0.061125,0.043685,0.239256,{870},"{707, 708, 711}"


In [37]:
new_filtered_rules = filtered_rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']]

new_filtered_rules['antecedents'] = new_filtered_rules['antecedents'].apply(extract_product_ids)
new_filtered_rules['consequents'] = new_filtered_rules['consequents'].apply(extract_product_ids)

def replace_ids_with_names(itemset):
    return [product_id_to_name.get(item, item) for item in itemset]

# Apply the function to the antecedents and consequents columns
new_filtered_rules['antecedents'] = new_filtered_rules['antecedents'].apply(replace_ids_with_names)
new_filtered_rules['consequents'] = new_filtered_rules['consequents'].apply(replace_ids_with_names)

new_filtered_rules['antecedents'] = new_filtered_rules['antecedents'].apply(lambda x: '; '.join(list(x)))
new_filtered_rules['consequents'] = new_filtered_rules['consequents'].apply(lambda x: '; '.join(list(x)))

new_filtered_rules
new_filtered_rules.to_sql('InventoryRule', engine, if_exists='append', index=False)

C:\Users\lenovo\AppData\Local\Temp\ipykernel_20132\1404364021.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_filtered_rules['antecedents'] = new_filtered_rules['antecedents'].apply(extract_product_ids)
C:\Users\lenovo\AppData\Local\Temp\ipykernel_20132\1404364021.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_filtered_rules['consequents'] = new_filtered_rules['consequents'].apply(extract_product_ids)
C:\Users\lenovo\AppData\Local\Temp\ipykernel_20132\1404364021.py:10: SettingWithCopyWarnin

179